In [10]:
import pygame
import cv2
import mediapipe as mp
import os
import random
import numpy as np

pygame.font.init()

#Global variables
s_width = 1280
s_height = 960
play_width = 1280  # meaning 300 // 10 = 30 width per block
play_height = 960  # meaning 600 // 20 = 30 height per block
top_left_x = (s_width - play_width) // 2
top_left_y = s_height - play_height - 10

#put a text in the middle of the screen
def draw_text_middle(surface, text, size, color):
    font = pygame.font.SysFont("britannic", size, bold=True)
    label = font.render(text, 1, color)
    surface.blit(label, (top_left_x + play_width /2 - (label.get_width()/2), top_left_y + play_height/2 - label.get_height()/2))
    
def add_grid(img,step):
    h,w,n = img.shape
    a=0
    b=0
    img[a:h:step,:,:] = 0
    img[:,b:w:step,:] = 0
    return img

CAM = []
def main_game_tetris(win,a,b):
    V = ['v1.mp4','v2.mp4','v3.mp4','v4.mp4'] # videostreams
    background = np.zeros([480*a, 640*b,3]) # to concatenate sterams
    zero_frame = np.zeros([480,640,3])
    clock = pygame.time.Clock() # create a timer
    FPS = 30 # set FPS

    # create virtual cameras
    cam = cv2.VideoCapture(0) # directly from camera
    for i in range(4):
#         cam = cv2.VideoCapture(V[i]) # from video
        CAM.append(cam)

    all_streams_are_available = True
    RUN = True
    if all_streams_are_available:
        # run the game
        while RUN:
            # read frame from each camera
            IMG = []
            for i in range(4):
                success, frame = CAM[i].read()
                # flip frame horizontally
                frame = cv2.flip(frame,1)
                if success:
                    dim = (640, 480)
                    # resize
                    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
                else:
                    frame = zero_frame
                # save frames
                frame = add_grid(frame,step=40)
                IMG.append(frame)
                
            # here supposed to be separate stream processing:
            # gesture reckognition
            # game interface overwriting

            # concatenate all frames on the background
            c = 0 # counter
            for i in range(a):
                for j in range(b):
                    if c > n:
                        break
                    background[480*i:480*(i+1),640*j:640*(j+1),:] = IMG[c]
                    c+=1

            # create surface
            surf = pygame.surfarray.make_surface(background.swapaxes(0,1))
            win.blit(surf,(0,0))
            pygame.display.update()

            clock.tick(FPS)

            # exit the game
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    print('EXIT the GAME')
                    RUN = False
#                     pygame.quit()

def main_menu_tetris(win,a,b):
    run = True
    while run:
        win.fill((0,0,0))
        draw_text_middle(win, 'Press Any Key To Start', 60, (255,255,255))
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                print('CLOSE the GAME')
            if event.type == pygame.KEYDOWN:
                print('START')
                main_game_tetris(win,a,b)
    pygame.display.quit()

# window properties
n = 4
if n == 1:
    a,b = 1,1
elif n == 2:
    a,b = 2,1
elif n >= 3 and n <=4:
    a,b = 2,2
elif n >=5 and n <= 6:
    a,b = 2,3
else:
    None

# create window
win = pygame.display.set_mode((640*b, 480*a))
# run main menu
main_menu_tetris(win,a,b)

START
EXIT the GAME
CLOSE the GAME
